In [137]:
import pandas as pd
from pathlib import Path

# Set the option to display all columns
pd.set_option('display.max_columns', None)

# Table of Contents
- [Setup](##Setup)
- [VIOLATION_CODE Preparation](#violation_code-preparation)
- [ISSUING_AGENCY_NAME Preparation](#issuing_agency_name-preparation)
- [ISSUE_TIME Preparation](#issue_time-preparation)
- [New Boolean Weekday or Weekend Column](#New-Boolean-Weekday-or-Weekend-Column)

## Setup

In [138]:
data = Path("Resources/tickets_2024_09.csv")
df = pd.read_csv(data)

C:\Users\rfqwx\AppData\Local\Temp\ipykernel_32068\818439906.py:2: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data)


In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98493 entries, 0 to 98492
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   OBJECTID              98493 non-null  int64  
 1   TICKET_NUMBER         98493 non-null  int64  
 2   VIOLATION_TYPE_DESC   98493 non-null  object 
 3   ISSUE_DATE            98493 non-null  object 
 4   ISSUE_TIME            98493 non-null  int64  
 5   ISSUING_AGENCY_CODE   98493 non-null  int64  
 6   ISSUING_AGENCY_NAME   98493 non-null  object 
 7   ISSUING_AGENCY_SHORT  98493 non-null  object 
 8   VIOLATION_CODE        98493 non-null  object 
 9   VIOLATION_PROC_DESC   98485 non-null  object 
 10  LOCATION              98493 non-null  object 
 11  PLATE_STATE           98493 non-null  object 
 12  VEHICLE_TYPE          0 non-null      float64
 13  MULTI_OWNER_NUMBER    98493 non-null  int64  
 14  DISPOSITION_CODE      98493 non-null  int64  
 15  DISPOSITION_TYPE   

In [140]:
df

,OBJECTID,TICKET_NUMBER,VIOLATION_TYPE_DESC,ISSUE_DATE,ISSUE_TIME,ISSUING_AGENCY_CODE,ISSUING_AGENCY_NAME,ISSUING_AGENCY_SHORT,VIOLATION_CODE,VIOLATION_PROC_DESC,LOCATION,PLATE_STATE,VEHICLE_TYPE,MULTI_OWNER_NUMBER,DISPOSITION_CODE,DISPOSITION_TYPE,DISPOSITION_DESC,DISPOSITION_DATE,FINE_AMOUNT,TOTAL_PAID,PENALTY_1,PENALTY_2,PENALTY_3,PENALTY_4,PENALTY_5,XCOORD,YCOORD,LATITUDE,LONGITUDE,MAR_ID,GIS_LAST_MOD_DTTM
0,83487425,260566401,P,2024/09/05 04:00:00+00,657,54,ST.ELZBETH HOSPITAL SECURITY GUARDS,SEH,P170,FAILURE TO DISPLAY CURRENT TAGS,FRONT NEW HOSPITAL SE,,NaN,0,0,Other,NaN,NaN,100,0,100,0,0,0,0,NaN,NaN,NaN,NaN,NaN,2024/10/16 14:04:18+00
1,83487426,266969312,P,2024/09/18 04:00:00+00,133,59,US. BUREAU OF ENGRAVING AND PRINTNG,BEP,P055,NO PARKING ANYTIME,ES D STREET SW,,NaN,0,0,Other,NaN,NaN,30,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,2024/10/16 14:04:18+00
2,83487445,266974724,P,2024/09/01 04:00:00+00,1020,59,US. BUREAU OF ENGRAVING AND PRINTNG,BEP,P012,DISOBEYING OFFICIAL SIGN,BLK ES 300 RAOUL WALLENBERG PL SW,,NaN,0,0,Other,NaN,NaN,30,30,0,0,0,0,0,397076.780,135370.390,38.886,-77.034,810050.0,2024/10/16 14:04:18+00
3,83487446,266974735,P,2024/09/08 04:00:00+00,1051,59,US. BUREAU OF ENGRAVING AND PRINTNG,BEP,P170,FAILURE TO DISPLAY CURRENT TAGS,SS 1300 BLK C ST SW,,NaN,0,106,Dismissed,DISM NOI SUBM LATE,2024/09/28 04:00:00+00,100,0,0,0,0,0,0,397328.708,135348.587,38.886,-77.031,805720.0,2024/10/16 14:04:18+00
4,83487447,266974746,P,2024/09/08 04:00:00+00,1108,59,US. BUREAU OF ENGRAVING AND PRINTNG,BEP,P170,FAILURE TO DISPLAY CURRENT TAGS,NS 1300 BLK D ST SW,,NaN,0,106,Dismissed,DISM NOI SUBM LATE,2024/09/28 04:00:00+00,100,0,0,0,0,0,0,397328.637,135237.646,38.885,-77.031,801948.0,2024/10/16 14:04:18+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98488,83587778,8271938231,P,2024/09/30 04:00:00+00,1535,15,DEPARTMENT OF PUBLIC WORKS,DPW,P012,DISOBEYING OFFICIAL SIGN,400 BLOCK NEW JERSEY AVE SE SOU*,,NaN,0,0,Other,NaN,NaN,30,0,0,0,0,0,0,399383.850,135200.010,38.885,-77.007,812200.0,2024/10/16 14:04:59+00
98489,83587779,8271938242,P,2024/09/30 04:00:00+00,1638,15,DEPARTMENT OF PUBLIC WORKS,DPW,P259,NO STOPPING OR STANDING IN PM RUSH HOUR ZONE,400 BLOCK NORTH CAROLINA AVE SE*,,NaN,0,0,Other,NaN,NaN,100,0,0,0,0,0,0,399995.838,135331.512,38.886,-77.000,803850.0,2024/10/16 14:04:59+00
98490,83587780,8271938253,P,2024/09/30 04:00:00+00,1639,15,DEPARTMENT OF PUBLIC WORKS,DPW,P170,FAILURE TO DISPLAY CURRENT TAGS,400 BLOCK NORTH CAROLINA AVE SE*,,NaN,0,0,Other,NaN,NaN,100,0,0,0,0,0,0,399995.838,135331.512,38.886,-77.000,803850.0,2024/10/16 14:04:59+00
98491,83587781,8271938264,P,2024/09/30 04:00:00+00,1640,15,DEPARTMENT OF PUBLIC WORKS,DPW,P259,NO STOPPING OR STANDING IN PM RUSH HOUR ZONE,400 BLOCK NORTH CAROLINA AVE SE*,,NaN,0,0,Other,NaN,NaN,100,0,0,0,0,0,0,399995.838,135331.512,38.886,-77.000,803850.0,2024/10/16 14:04:59+00


## VIOLATION_CODE Preparation
VIOLATION_CODE contains the designation for the specific type of violation committed.
We will combine all VIOLATION_CODE values that occurr less than 30 times (less than once per day on average) into a single code.
[Go to Table of Contents](#table-of-contents)

In [141]:
new_df = df.copy()

In [142]:
# Count unique values
violation_codes = new_df["VIOLATION_CODE"].value_counts()

# Create dataframe
violation_df = violation_codes.reset_index()
violation_df.columns = ["VIOLATION_CODE", "count"]

# Calculate cumulative percentage 
violation_df["percent"] = (violation_df["count"] / len(df)) * 100
violation_df["cumulative_percent"] = violation_df["percent"].cumsum()

# Display all codes
with pd.option_context('display.max_rows', None):
    print(violation_df)

    VIOLATION_CODE  count    percent  cumulative_percent
0             P281  12465  12.655722           12.655722
1             P012   8841   8.976272           21.631994
2             P173   8784   8.918400           30.550394
3             P170   8278   8.404658           38.955053
4             P055   5809   5.897881           44.852934
5             P269   4974   5.050105           49.903039
6             P259   4335   4.401328           54.304367
7             P003   4330   4.396252           58.700618
8             P168   3761   3.818545           62.519164
9             P076   3006   3.051994           65.571157
10            P039   2772   2.814413           68.385571
11            P031   2508   2.546374           70.931944
12            P322   2421   2.458043           73.389987
13            P159   2373   2.409308           75.799295
14            P172   1959   1.988974           77.788269
15            P113   1465   1.487415           79.275685
16            P344   1273   1.2

In [143]:
# COMBINE ALL CODES OCCURRING LESS THAN 30 TIMES INTO A SINGLE AGGREGATE CODE

# Identify all codes occurring less than 30 times
violation_codes_to_replace = violation_codes[violation_codes < 30].index.tolist()

# Replace in dataframe
for code in violation_codes_to_replace:
    new_df['VIOLATION_CODE'] = new_df['VIOLATION_CODE'].replace(code, "OTHER")

# Check to make sure replacement was successful
new_df['VIOLATION_CODE'].value_counts()


VIOLATION_CODE
P281    12465
P012     8841
P173     8784
P170     8278
P055     5809
        ...  
P004       47
P278       43
P011       42
P034       40
P025       35
Name: count, Length: 72, dtype: int64

## ISSUING_AGENCY_NAME Preparation
Combine agencies that issue fewer than 30 tickets into one code.
[Go to Table of Contents](#table-of-contents)

In [144]:
issuing_agencies = new_df["ISSUING_AGENCY_NAME"].value_counts()
issuing_agencies

ISSUING_AGENCY_NAME
DEPARTMENT OF PUBLIC WORKS             94235
DDOT                                    2106
METROPOLITAN POLICE DPT-DISTRICT 1       508
UNITED STATES CAPITOL POLICE             396
TAXI COMMISSION                          259
METROPOLITAN POLICE DPT-DISTRICT 3       193
METROPOLITAN POLICE DPT-DISTRICT 2       151
METROPOLITAN POLICE DPT-DISTRICT 4       113
METRO POLICE                              95
METROPOLITAN POLICE DPT-DISTRICT 5        94
US. BUREAU OF ENGRAVING AND PRINTNG       69
SPECIAL OPERATION DIV & TRAFFIC DIV       57
METROPOLITAN POLICE DPT-DISTRICT 6        49
UNITED STATES PARK POLICE                 44
METROPOLITAN POLICE DPT-DISTRICT 7        32
UNVRSTY OF THE D.C. SECURITY POLICE       20
US. SECRET SERVICE UNIFORM DIVISION       18
BOLLING AFB                               10
MPD RESERVE CORPS                          8
PROTECTIVE SERVICES DEPT (DC GOVT)         6
FBI                                        5
METRO POLICE DPT-PROPERTY DIVISION 

In [145]:
# COMBINE ALL NAMES OCCURRING LESS THAN 30 TIMES INTO A SINGLE AGGREGATE NAME

# Identify all names occurring less than 30 times
issuing_agencies_to_replace = issuing_agencies[issuing_agencies < 30].index.tolist()

# Replace in dataframe
for name in issuing_agencies_to_replace:
    new_df['ISSUING_AGENCY_NAME'] = new_df['ISSUING_AGENCY_NAME'].replace(name, "OTHER")

In [146]:
# Check to make sure replacement was successful
new_df['ISSUING_AGENCY_NAME'].value_counts()

ISSUING_AGENCY_NAME
DEPARTMENT OF PUBLIC WORKS             94235
DDOT                                    2106
METROPOLITAN POLICE DPT-DISTRICT 1       508
UNITED STATES CAPITOL POLICE             396
TAXI COMMISSION                          259
METROPOLITAN POLICE DPT-DISTRICT 3       193
METROPOLITAN POLICE DPT-DISTRICT 2       151
METROPOLITAN POLICE DPT-DISTRICT 4       113
METRO POLICE                              95
METROPOLITAN POLICE DPT-DISTRICT 5        94
OTHER                                     92
US. BUREAU OF ENGRAVING AND PRINTNG       69
SPECIAL OPERATION DIV & TRAFFIC DIV       57
METROPOLITAN POLICE DPT-DISTRICT 6        49
UNITED STATES PARK POLICE                 44
METROPOLITAN POLICE DPT-DISTRICT 7        32
Name: count, dtype: int64

## ISSUE_TIME Preparation
Turn this numeric column into a categorical column based on the period of the day: late night, morning commute, morning, lunch, afternoon, evening commute, and night.[Go to Table of Contents](#table-of-contents)

In [147]:
# Define a function to map ISSUE_TIME to periods of the day
def categorize_time(time):
    if time > 2300 or time < 700:  # Late Night/Early Morning
        return 0
    elif 700 <= time < 900:  # Morning Commute
        return 1
    elif 900 <= time < 1100:  # Morning
        return 2
    elif 1100 <= time < 1300:  # Lunch
        return 3
    elif 1300 <= time < 1700:  # Afternoon
        return 4
    elif 1700 <= time < 2000:  # Evening Commute
        return 5
    else:  # Night
        return 6

# Apply the function to the ISSUE_TIME column, then insert the new column immediately after the ISSUE_TIME column
new_df['PERIOD'] = new_df['ISSUE_TIME'].apply(categorize_time)
issue_time_index = new_df.columns.get_loc('ISSUE_TIME')
new_df.insert(issue_time_index + 1, 'PERIOD', new_df.pop('PERIOD'))
new_df


,OBJECTID,TICKET_NUMBER,VIOLATION_TYPE_DESC,ISSUE_DATE,ISSUE_TIME,PERIOD,ISSUING_AGENCY_CODE,ISSUING_AGENCY_NAME,ISSUING_AGENCY_SHORT,VIOLATION_CODE,VIOLATION_PROC_DESC,LOCATION,PLATE_STATE,VEHICLE_TYPE,MULTI_OWNER_NUMBER,DISPOSITION_CODE,DISPOSITION_TYPE,DISPOSITION_DESC,DISPOSITION_DATE,FINE_AMOUNT,TOTAL_PAID,PENALTY_1,PENALTY_2,PENALTY_3,PENALTY_4,PENALTY_5,XCOORD,YCOORD,LATITUDE,LONGITUDE,MAR_ID,GIS_LAST_MOD_DTTM
0,83487425,260566401,P,2024/09/05 04:00:00+00,657,0,54,OTHER,SEH,P170,FAILURE TO DISPLAY CURRENT TAGS,FRONT NEW HOSPITAL SE,,NaN,0,0,Other,NaN,NaN,100,0,100,0,0,0,0,NaN,NaN,NaN,NaN,NaN,2024/10/16 14:04:18+00
1,83487426,266969312,P,2024/09/18 04:00:00+00,133,0,59,US. BUREAU OF ENGRAVING AND PRINTNG,BEP,P055,NO PARKING ANYTIME,ES D STREET SW,,NaN,0,0,Other,NaN,NaN,30,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,2024/10/16 14:04:18+00
2,83487445,266974724,P,2024/09/01 04:00:00+00,1020,2,59,US. BUREAU OF ENGRAVING AND PRINTNG,BEP,P012,DISOBEYING OFFICIAL SIGN,BLK ES 300 RAOUL WALLENBERG PL SW,,NaN,0,0,Other,NaN,NaN,30,30,0,0,0,0,0,397076.780,135370.390,38.886,-77.034,810050.0,2024/10/16 14:04:18+00
3,83487446,266974735,P,2024/09/08 04:00:00+00,1051,2,59,US. BUREAU OF ENGRAVING AND PRINTNG,BEP,P170,FAILURE TO DISPLAY CURRENT TAGS,SS 1300 BLK C ST SW,,NaN,0,106,Dismissed,DISM NOI SUBM LATE,2024/09/28 04:00:00+00,100,0,0,0,0,0,0,397328.708,135348.587,38.886,-77.031,805720.0,2024/10/16 14:04:18+00
4,83487447,266974746,P,2024/09/08 04:00:00+00,1108,3,59,US. BUREAU OF ENGRAVING AND PRINTNG,BEP,P170,FAILURE TO DISPLAY CURRENT TAGS,NS 1300 BLK D ST SW,,NaN,0,106,Dismissed,DISM NOI SUBM LATE,2024/09/28 04:00:00+00,100,0,0,0,0,0,0,397328.637,135237.646,38.885,-77.031,801948.0,2024/10/16 14:04:18+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98488,83587778,8271938231,P,2024/09/30 04:00:00+00,1535,4,15,DEPARTMENT OF PUBLIC WORKS,DPW,P012,DISOBEYING OFFICIAL SIGN,400 BLOCK NEW JERSEY AVE SE SOU*,,NaN,0,0,Other,NaN,NaN,30,0,0,0,0,0,0,399383.850,135200.010,38.885,-77.007,812200.0,2024/10/16 14:04:59+00
98489,83587779,8271938242,P,2024/09/30 04:00:00+00,1638,4,15,DEPARTMENT OF PUBLIC WORKS,DPW,P259,NO STOPPING OR STANDING IN PM RUSH HOUR ZONE,400 BLOCK NORTH CAROLINA AVE SE*,,NaN,0,0,Other,NaN,NaN,100,0,0,0,0,0,0,399995.838,135331.512,38.886,-77.000,803850.0,2024/10/16 14:04:59+00
98490,83587780,8271938253,P,2024/09/30 04:00:00+00,1639,4,15,DEPARTMENT OF PUBLIC WORKS,DPW,P170,FAILURE TO DISPLAY CURRENT TAGS,400 BLOCK NORTH CAROLINA AVE SE*,,NaN,0,0,Other,NaN,NaN,100,0,0,0,0,0,0,399995.838,135331.512,38.886,-77.000,803850.0,2024/10/16 14:04:59+00
98491,83587781,8271938264,P,2024/09/30 04:00:00+00,1640,4,15,DEPARTMENT OF PUBLIC WORKS,DPW,P259,NO STOPPING OR STANDING IN PM RUSH HOUR ZONE,400 BLOCK NORTH CAROLINA AVE SE*,,NaN,0,0,Other,NaN,NaN,100,0,0,0,0,0,0,399995.838,135331.512,38.886,-77.000,803850.0,2024/10/16 14:04:59+00


## New Boolean Weekday or Weekend Column
[Go to Table of Contents](#table-of-contents)

In [148]:
# Convert ISSUE_DATE to datetime format
new_df['ISSUE_DATE'] = pd.to_datetime(new_df['ISSUE_DATE'])

# Add a new column: 0 for weekday, 1 for weekend
new_df['WEEKEND'] = new_df['ISSUE_DATE'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)
issue_date_index = new_df.columns.get_loc('ISSUE_DATE')
new_df.insert(issue_date_index + 1, 'WEEKEND', new_df.pop('WEEKEND'))

In [149]:
new_df

,OBJECTID,TICKET_NUMBER,VIOLATION_TYPE_DESC,ISSUE_DATE,WEEKEND,ISSUE_TIME,PERIOD,ISSUING_AGENCY_CODE,ISSUING_AGENCY_NAME,ISSUING_AGENCY_SHORT,VIOLATION_CODE,VIOLATION_PROC_DESC,LOCATION,PLATE_STATE,VEHICLE_TYPE,MULTI_OWNER_NUMBER,DISPOSITION_CODE,DISPOSITION_TYPE,DISPOSITION_DESC,DISPOSITION_DATE,FINE_AMOUNT,TOTAL_PAID,PENALTY_1,PENALTY_2,PENALTY_3,PENALTY_4,PENALTY_5,XCOORD,YCOORD,LATITUDE,LONGITUDE,MAR_ID,GIS_LAST_MOD_DTTM
0,83487425,260566401,P,2024-09-05 04:00:00+00:00,0,657,0,54,OTHER,SEH,P170,FAILURE TO DISPLAY CURRENT TAGS,FRONT NEW HOSPITAL SE,,NaN,0,0,Other,NaN,NaN,100,0,100,0,0,0,0,NaN,NaN,NaN,NaN,NaN,2024/10/16 14:04:18+00
1,83487426,266969312,P,2024-09-18 04:00:00+00:00,0,133,0,59,US. BUREAU OF ENGRAVING AND PRINTNG,BEP,P055,NO PARKING ANYTIME,ES D STREET SW,,NaN,0,0,Other,NaN,NaN,30,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,2024/10/16 14:04:18+00
2,83487445,266974724,P,2024-09-01 04:00:00+00:00,1,1020,2,59,US. BUREAU OF ENGRAVING AND PRINTNG,BEP,P012,DISOBEYING OFFICIAL SIGN,BLK ES 300 RAOUL WALLENBERG PL SW,,NaN,0,0,Other,NaN,NaN,30,30,0,0,0,0,0,397076.780,135370.390,38.886,-77.034,810050.0,2024/10/16 14:04:18+00
3,83487446,266974735,P,2024-09-08 04:00:00+00:00,1,1051,2,59,US. BUREAU OF ENGRAVING AND PRINTNG,BEP,P170,FAILURE TO DISPLAY CURRENT TAGS,SS 1300 BLK C ST SW,,NaN,0,106,Dismissed,DISM NOI SUBM LATE,2024/09/28 04:00:00+00,100,0,0,0,0,0,0,397328.708,135348.587,38.886,-77.031,805720.0,2024/10/16 14:04:18+00
4,83487447,266974746,P,2024-09-08 04:00:00+00:00,1,1108,3,59,US. BUREAU OF ENGRAVING AND PRINTNG,BEP,P170,FAILURE TO DISPLAY CURRENT TAGS,NS 1300 BLK D ST SW,,NaN,0,106,Dismissed,DISM NOI SUBM LATE,2024/09/28 04:00:00+00,100,0,0,0,0,0,0,397328.637,135237.646,38.885,-77.031,801948.0,2024/10/16 14:04:18+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98488,83587778,8271938231,P,2024-09-30 04:00:00+00:00,0,1535,4,15,DEPARTMENT OF PUBLIC WORKS,DPW,P012,DISOBEYING OFFICIAL SIGN,400 BLOCK NEW JERSEY AVE SE SOU*,,NaN,0,0,Other,NaN,NaN,30,0,0,0,0,0,0,399383.850,135200.010,38.885,-77.007,812200.0,2024/10/16 14:04:59+00
98489,83587779,8271938242,P,2024-09-30 04:00:00+00:00,0,1638,4,15,DEPARTMENT OF PUBLIC WORKS,DPW,P259,NO STOPPING OR STANDING IN PM RUSH HOUR ZONE,400 BLOCK NORTH CAROLINA AVE SE*,,NaN,0,0,Other,NaN,NaN,100,0,0,0,0,0,0,399995.838,135331.512,38.886,-77.000,803850.0,2024/10/16 14:04:59+00
98490,83587780,8271938253,P,2024-09-30 04:00:00+00:00,0,1639,4,15,DEPARTMENT OF PUBLIC WORKS,DPW,P170,FAILURE TO DISPLAY CURRENT TAGS,400 BLOCK NORTH CAROLINA AVE SE*,,NaN,0,0,Other,NaN,NaN,100,0,0,0,0,0,0,399995.838,135331.512,38.886,-77.000,803850.0,2024/10/16 14:04:59+00
98491,83587781,8271938264,P,2024-09-30 04:00:00+00:00,0,1640,4,15,DEPARTMENT OF PUBLIC WORKS,DPW,P259,NO STOPPING OR STANDING IN PM RUSH HOUR ZONE,400 BLOCK NORTH CAROLINA AVE SE*,,NaN,0,0,Other,NaN,NaN,100,0,0,0,0,0,0,399995.838,135331.512,38.886,-77.000,803850.0,2024/10/16 14:04:59+00


## PLATE_STATE Preparation
Replace blank values with DC. Aggregate states that appear infrequently into a single category. [Go to Table of Contents](#table-of-contents)

In [150]:
plate_states = new_df["PLATE_STATE"].value_counts()
plate_states

PLATE_STATE
      96742
VA     1265
GA      126
FL      107
MD       99
IL       59
WA       24
MI       22
AL        9
NC        8
NJ        7
MO        5
SC        4
TX        3
WI        2
DC        2
WV        1
VT        1
CO        1
LA        1
NV        1
MN        1
OK        1
IN        1
OH        1
Name: count, dtype: int64

In [151]:
# Identify states whose plates were ticketed less than 50 times (i.e., less than approximately 0.05% of all tickets)
plate_states_to_replace = plate_states[plate_states < 50].index.tolist()
plate_states_to_replace

# Replace in dataframe
for state in plate_states_to_replace:
    new_df['PLATE_STATE'] = new_df['PLATE_STATE'].replace(state, "OTHER")

In [152]:
# Replace null and single-space values with DC. Rows for DC vehicles seem to have a single space as value.
new_df['PLATE_STATE'] = new_df['PLATE_STATE'].astype(str)
new_df['PLATE_STATE'] = new_df['PLATE_STATE'].replace(['', ' ', 'nan', 'None'], 'DC')
new_df['PLATE_STATE'] = new_df['PLATE_STATE'].fillna('DC')


In [153]:
# Check to make sure replacement was successful
new_df['PLATE_STATE'].value_counts()

PLATE_STATE
DC       96742
VA        1265
GA         126
FL         107
MD          99
OTHER       95
IL          59
Name: count, dtype: int64

In [156]:
# Export new_df to csv
new_df.to_csv('Resources/updated_tickets_2024_09.csv', index=False)